# Gradio Brochure Generator

We will build a User Interface for generating a company brochure using the HuggingFace Gradio framework.

In [1]:
# imports
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
import gradio as gr # Gradio for UI

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCG


In [4]:
# Connect to OpenAI, Anthropic and Google; 

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [5]:
# System message

system_message = "You are a helpful assistant"

In [6]:
# Call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

## Forcing dark mode

Gradio appears in light mode or dark mode depending on the settings of the browser and computer.

In [8]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [9]:
# Build interface for "message_gpt"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


In [13]:
# Let's use Markdown
# I'm taking advantage of system_message being a global variable, used back in the message_gpt function 

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


In [15]:
# Call that streams back results

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        result = result.replace("```","").replace("markdown","")
        update_display(Markdown(result), display_id=display_handle.display_id)
        yield result

In [16]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


## Call to Claude 

In [17]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [18]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.


In [19]:
import google.generativeai
#from google import genai

In [20]:
google.generativeai.configure()
#client = genai.Client(api_key=google_api_key)

## Call to Gemini

In [23]:
def stream_gemini(prompt):
    stream = google.generativeai.GenerativeModel(
        model_name='gemini-2.0-flash',
        system_instruction=system_message,
        generation_config={"response_mime_type": "text/markdown"}
    )
    response = stream.generate_content(prompt, stream=True)
    result = ""
    for chunk in response:
        # For streaming, content is directly in `chunk.text` or `chunk.candidates[0].content.parts.text`
        if hasattr(chunk, 'text') and chunk.text:
            result+= chunk.text or ""
            yield result
        elif hasattr(chunk, 'candidates') and chunk.candidates:
            parts = getattr(chunk.candidates[0].content, "parts", [])
            piece = "".join(getattr(p, "text", "") for p in parts if getattr(p, "text", ""))
            if piece:
                result += piece
                yield result


In [24]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [25]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7881
* To create a public link, set `share=True` in `launch()`.


# Building the company brochure generator

In [26]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [27]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [28]:
def stream_brochure(company_name, url, model, tone):
    if tone=="Humorous":
        prompt = f"Please generate a humorous and playfull company brochure for {company_name}. Here is their landing page:\n"
        prompt += Website(url).get_contents()
    elif tone=="Serious":
        prompt = f"Please generate a serious company brochure for {company_name}. Here is their landing page:\n"
        prompt += Website(url).get_contents()
    elif tone=="Sarcastic":
        prompt = f"Please generate a sarcastic and mean company brochure for {company_name}. Here is their landing page:\n"
        prompt += Website(url).get_contents()
    else:
        raise ValueError("Unknown tone")

    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Gemini":
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [29]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model"),
        gr.Dropdown(["Humorous", "Serious", "Sarcastic"], label="Select tone")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.
